## Calling packages and assigning variables
Here i call the necessary packages as well as assigning variables. Of note are the paths to collected data which will need to be changed for replication in another system

In [ ]:
import pandas as pd
import rdkit
import numpy as np
import pickle
import os
import gc
from dask.distributed import Client
from tpot import TPOTClassifier
from rdkit import Chem
from rdkit.Chem import MACCSkeys
from mordred import Calculator, descriptors
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

try:
    import misvm 
except:
    !pip install -e git+https://github.com/garydoranjr/misvm.git#egg=misvm
    print("\n \n Install complete, please restart kernal")

gc.enable()

# Cleaning Biotransformer data
This section is used to transform the input data such that it is usable for model building. This involves matching metabolite to parent molecules as well as finding
thier canonical smiles. <br /><br />
Additionally this section pre-calculates the encoding of each molecule for modeling to save on time, especially for subsequent runs

In [ ]:
def normalize_smiles(smi):      ## Converts each SMILES to an RDkit molecule then reconverts to SMILES. Ensures molecules with the same structure are the same SMILES
    try:
        smi_norm = Chem.MolToSmiles(Chem.MolFromSmiles(smi))
        return smi_norm
    except:
        return np.nan

def parent_finder(smi):         ## Searches the "smiles" column of the "data" dataframe of the original molecules and normalises both, returning a match if found
    for parent in data['smiles']:
        try:
            if Chem.MolToSmiles(Chem.MolFromSmiles(smi)) == Chem.MolToSmiles(Chem.MolFromSmiles(parent)):
                return parent
        except:
            continue
    return "No parent found"

def get_ml_encoding(df, function=MACCSkeys.GenMACCSKeys):   ## returns a list, where each . Done to whole dataframe to allow for removing columns with NaN values
    def number_check(x):            ## Checks if a value can be converted to a float. Used to remove non-numeric encoding
        try:
            float(x)
            return x
        except:
            return "broken"

    ## Generate encoding list from smiles
    working_df = df.copy()                                                                     
    working_df['encoding_list'] = working_df['smiles'].apply(lambda x: list(function(Chem.MolFromSmiles(x))))   
    ## Create dataframe from encoding list
    encoding_df = pd.DataFrame(working_df['encoding_list'].to_list())   
    ## Cleaning Nan or non-numeric columns, as well as removing single value columns
    encoding_df = encoding_df.applymap(number_check).dropna(axis =1, how = "any")  
    encoding_df = encoding_df.drop(columns=encoding_df.columns[(encoding_df == 'broken').any()])
    encoding_df = encoding_df[[c for c in list(encoding_df) if len(encoding_df[c].unique()) > 1]]                                           
    ## Transform encoding dataframe into a list of lists. Can be used to generate a new column of the original dataframe                       
    X = encoding_df.values.tolist()                                                                                                                       
    return X

def bag_parent(smiles,met_df,function):
    ## Create dataframe from molecules with parent == smiles as well as the smiles molecule. Removes duplicates
    mol_family          =   met_df[met_df["parent smiles"]==smiles].append({'smiles':smiles},ignore_index=True).drop_duplicates(subset=["smiles"])
    ##  Encodes this family using above
    mol_family_encoded  =   get_ml_encoding(df = mol_family, function = function)
    return mol_family_encoded

In [ ]:
if os.path.isfile("encoded_data.pk1"):      ##  Checks if file already exists. Saves time and no changes are expected
    print("Data already encoded")

else:
    ##          Step 1: Load data into dataframes
    data = pd.read_csv("selected_molecules.csv")
    metabolite_data = pd.read_csv("biotransformer_output_cyp1.csv").append(pd.read_csv("biotransformer_output_phaseII.csv"))

    ##          Step 2: Normailizing metabolite smiles and matching to parent (approx 220 secs) 
    metabolite_data['smiles']           = metabolite_data['SMILES'].apply(lambda x: normalize_smiles(x)).dropna(axis=0,subset=['smiles'])
    metabolite_data['parent smiles']    = metabolite_data['Precursor SMILES'].apply(lambda x:parent_finder(x))

    ##          Step 3: Pre calculating encoding for molecules, requires evaluation of lists on loading csv (approx 110 secs)
    data["MACCS"] = get_ml_encoding(df = data, function = MACCSkeys.GenMACCSKeys)
    data["RDKF"] = get_ml_encoding(df = data, function =  Chem.RDKFingerprint)
    data["MACCS_MIL"] = data.apply(lambda row: bag_parent(smiles = row['smiles'], met_df = metabolite_data, function = MACCSkeys.GenMACCSKeys),axis=1)
    data["RDKF_MIL"] = data.apply(lambda row: bag_parent(smiles = row['smiles'], met_df = metabolite_data, function = Chem.RDKFingerprint),axis=1)
    
    ##          Step 3: Saved to a pickle, rather than a csv this stores the lists and is much faster to load (~10x)
    data = data.drop(["Molecule"],axis=1)
    data.to_pickle("encoded_data.pk1")

Data already encoded


# Defining functions
This section is where i define functions for model development. <br /> If you're curious on how it is done please look here

In [ ]:
def check_if_tested(suffix,model_name,encoding):    ## Checking if this build/test has already been done. Saves on time if a run crashes
    if not os.path.isfile("total_results.pk1"): 
        already_complete = False
    else:
        results = pd.read_pickle("total_results.pk1")
        already_complete = ((results["fold"].isin([suffix["fold"]])) & (results["iteration"].isin([suffix["iteration"]])) & (results["model"].isin([model_name])) & (results["encoding"].isin([encoding]))).any()
    return already_complete

def build_test_mil_model(training_data,testing_data,MIL,encoding,suffix,save_model,model_name = ""):    ## Build and test a MIL model
    already_complete = check_if_tested(suffix=suffix,encoding=encoding,model_name=model_name)
    if not already_complete:
        ##      Building model, note encoding already performed
        bags = training_data[encoding+"_MIL"].to_list()
        labels = training_data["Ames"].apply(lambda x: x if x==1 else -1).to_list()
        model = MIL                                                              
        model.fit(bags,labels)    
        ##      Testing model
        bags = testing_data[encoding+"_MIL"].to_list()
        labels = testing_data["Ames"].apply(lambda x: x if x==1 else -1).to_list()
        predictions = model.predict(bags)                                        
        predicted_labels = list(map(pos_or_neg,predictions))                            
        df = pd.DataFrame({
            'predicted' : predictions,
            'predicted labal' : predicted_labels,
            'true label' : labels
        })  
        save_results(df = df, suffix = suffix, model = model_name, encoding = encoding)
        if save_model:
            save_models(model = model, path = "/saved_models/"+model_name+"_"+str(suffix["fold"])+"_"+str(suffix["iteration"]+".sav"))
    else:
        print("Already tested   fold:",suffix["fold"],"   iteration:",suffix["iteration"],"   model:",model_name,"   encoding:",encoding)

def build_test_ml_model(training_data,testing_data,encoding,ML,suffix,save_model):                      ## Build and test a machine learning model
    already_complete = check_if_tested(suffix=suffix,encoding=encoding,model_name="TPOT")
    if not already_complete:
        ##      Building model, note encoding already performed
        instances = np.array(training_data[encoding].to_list())
        labels = np.array(training_data["Ames"].to_list())       
        tpot_optimisation = ML                                                          
        tpot_optimisation.fit(instances,labels)    
        ##      Testing model
        model = tpot_optimisation.fitted_pipeline_  ## This takes the best fitted pipeline developed
        instances = testing_data[encoding].to_list()
        true_labels = testing_data["Ames"].to_list()       
        predictions = model.predict(instances)                                   
        predicted_labels = list(map(pos_or_neg,predictions))                            
        df = pd.DataFrame({
            'predicted' : predictions,
            'predicted labal' : predicted_labels,
            'true label' : true_labels
        })   
        save_results(df = df, suffix = suffix, model = "TPOT", encoding = encoding)  
        if save_model:
            save_models(model = model, path = "/saved_models/TPOT_"+str(suffix["fold"])+"_"+str(suffix["iteration"]+".sav"))
    else:
        print("Already tested   fold:",suffix["fold"],"   iteration:",suffix["iteration"],"   model:","TPOT","   encoding:",encoding)

def pos_or_neg(x):  ## Simple function used to translate predictions between MIL and ML into a single form
    if x>0:
        return 1
    else:
        return 0

def format_results(df,suffix,model,encoding):  ## adds informative columns to the df used for saving results 
    df["fold"]  =   suffix["fold"]
    df["iteration"] =   suffix["iteration"]
    df['index'] = df.index
    df["model"] =   model
    df["encoding"] =   encoding
    return df

def save_results(df,suffix,model,encoding):     ## saves results to a single pickle, adding to it or generating it
    if not os.path.isfile("total_results.pk1"):
        df_formatted = format_results(df=df,suffix=suffix,model=model,encoding=encoding)
        df_formatted.to_pickle("total_results.pk1")
    else:
        total_results = pd.read_pickle("total_results.pk1")
        df_formatted = format_results(df=df,suffix=suffix,model=model,encoding=encoding)
        total_results = total_results.append(df_formatted)
        total_results.to_pickle("total_results.pk1")
        
def save_models(model,path):                    ## Saves model to a path
    pickle.dump(model, open(path, 'wb'))

def develop_models(training_data,testing_data,suffix={"fold":"","iteration":""},encoding="MACCS",save_model=False, dask=False):     ## single function to complete whole pipeline for a set of data to all expected models
    ##      Step 0:     Checking that the encoding method described is expected
    fps = ["MACCS","RDFP"]
    if not encoding in fps:
        print('Please use expected encoding: ["MACCS", "RDFP"]')
        return
    
    ##      Step 1:     Model generation
    tested_mils =  [["MICA", misvm.MICA(max_iters=50,verbose=False)],     
                ["MISVM", misvm.MISVM(kernel='linear', C=1.0, max_iters=50,verbose=False)],
                ['SIL', misvm.SIL(verbose=False)],
                ['NSK', misvm.NSK(verbose=False)],
                ['sMIL', misvm.sMIL(verbose=False)]]
            ## note: Either as dask or non-dask TPOT can be used, defined in function variables
    if dask:
        tpot_model = TPOTClassifier(generations=10, population_size=500, cv=5, verbosity=2,use_dask=dask, n_jobs=-1)
    else:
        tpot_model = TPOTClassifier(generations=10, population_size=500, cv=5, verbosity=2, n_jobs=-1)

    ##      Step 2:     Build and test models
        # Iterate over the used MILs
    for mil in tested_mils:
        print("     Building and testing:",mil[0],"    fold:",suffix["fold"],"    Iteration:",suffix["iteration"])
        build_test_mil_model(training_data=training_data,testing_data=testing_data,suffix=suffix,MIL=mil[1],encoding=encoding,model_name=mil[0],save_model=save_model)
        # Build and test TPOT model
    print("     Building and testing: TPOT     fold:",suffix["fold"],"    Iteration:",suffix["iteration"])
    build_test_ml_model(training_data=training_data,testing_data=testing_data, ML = tpot_model,encoding=encoding,suffix=suffix,save_model=save_model)

## Setting dask false, means that you can just unhash below to enable it
dask = False

## Building models
Here the above functions are used to build models. This section can be altered to build additional models if desired

In [ ]:
# ## Setting up Dask to allow parrallel training. If you don't want this please hash this out and change "use_dask=True" to "use_dask=False" in the develop models function
# ################### #
# client = Client()   #
# client              #
# dask = 4            #
# ################### #

In [ ]:
## Selecting encoding method, can be changed to RDKF if desired
encoding = "MACCS"

##          Step 1: splitting data into a hold out validation dataset
training_data, test_data = train_test_split(pd.read_pickle("encoded_data.pk1"), test_size=0.2, stratify=pd.read_pickle("encoded_data.pk1")["Ames"], random_state=34783)
training_data = training_data.reset_index(drop=True);   test_data = test_data.reset_index(drop=True)

##          Step 2: Repeated stratified crossvalidation on training data
rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=6234794)
for fold,[train_index, validation_index] in enumerate(rskf.split(training_data, training_data["Ames"])):
    train   =   training_data.iloc[train_index]
    validation    =   training_data.iloc[validation_index]
    develop_models(training_data=train,testing_data=validation,encoding = encoding,suffix={"fold":fold%10,"iteration":fold//10},save_model=False,dask=dask)
    print("Done Fold", "    fold:",fold%10,"    iteration:",fold//10)
    gc.collect()

# ##          Step 3: model building on training data against holdout test data
# develop_models(training_data=train,testing_data=test_data,encoding = encoding,suffix={"fold":"","iteration":"Hold out test"},save_model=True)

# ## predict proba for some

     Building and testing: MICA     fold: 0     Iteration: 0


AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/home/samuel/anaconda3/envs/my-rdkit-env/lib/python3.9/site-packages/pandas/_libs/internals.cpython-39-x86_64-linux-gnu.so'>

## Model Validation
Here the model results are assessed

In [ ]:
# develop_models(data,bt_data,validation_data,validation_metabolite_data,suffix="_validation")

## Model Analysis
Here the results of each fold are calculated as well as deviation within crossvalidation

In [ ]:
def confusion_matrix(df):
    TP = len(df[(df["predicted label"] == 1) & (df["true label"] == 1)])
    TN = len(df[(df["predicted label"] == 0) & (df["true label"] == 0)])
    FP = len(df[(df["predicted label"] == 1) & (df["true label"] == 0)])
    FN = len(df[(df["predicted label"] == 0) & (df["true label"] == 1)])
    return [TP,TN,FP,FN]

def mean_accuracy(row):
    acc = (row["TP"]+row["TN"])/(row["TP"]+row["TN"]+row["FP"]+row["FN"])
    return acc

def mean_sensitivity(row):
    sens = row["TP"]/(row["TP"]+row["FP"])
    return sens

def mean_specificity(row):
    spec = row["TN"]/(row["TN"]+row["FN"])
    return spec

def mean_F1(row):
    f1 = (2*row["TP"])/(2*row["TP"]+row["FP"]+row["FN"])

In [ ]:
rslt_list = []

crossvalidation_results = pd.read_pickle("total_results.pk1")
for iteration in crossvalidation_results["iteration"].unique():
    for fold in crossvalidation_results["fold"].unique():
        for model in crossvalidation_results["model"].unique():
            for encoding in crossvalidation_results["encoding"].unique():
                working_data = crossvalidation_results[(crossvalidation_results["fold"]==fold)&(crossvalidation_results["iteration"]==iteration)&(crossvalidation_results["model"]==model)&(crossvalidation_results["encoding"]==encoding)]
                [TP,TN,FP,FN] = confusion_matrix(working_data)
                rslt_list += [{"encoding":encoding, "model":model, "fold":fold, "iteration":iteration, "TP":TP, "TN":TN, "FP":FP, "FN":FN}]
rslt_df = pd.Dataframe(rslt_list)

mean_rslt_list = []
for model in rslt_df["model"].unique():
    for encoding in rslt_df["encoding"].unique():
        working_data = rslt_df[(rslt_df["model"]==model)&(rslt_df["encoding"]==encoding)]
        mean_rslt_list += [{"encoding":encoding, "model":model, "Mean TP":working_data["TP"].mean(), "Mean TN":working_data["TN"].mean(), "Mean FP":working_data["FP"].mean(), "Mean FN":working_data["FN"].mean()}]
mean_rslt_df = pd.DataFrame(mean_rslt_list)
mean_rslt_df["accuracy"] = mean_rslt_df.apply(lambda x: mean_accuracy(x))
mean_rslt_df["sensitivity"] = mean_rslt_df.apply(lambda x: mean_sensitivity(x))
mean_rslt_df["specificity"] = mean_rslt_df.apply(lambda x: mean_specificity(x))
mean_rslt_df["F1"] = mean_rslt_df.apply(lambda x: mean_F1(x))
# mean_rslt_df["AUROC"] = mean_rslt_df.apply(lambda x: roc_auc_score())  #Need to redo with predict proba results


